# Script to Create Instructor in JupyterHub and nbgrader

In [ ]:
steps = [True, True, True, True]

In [ ]:
# Run this cell to create an instructor
import subprocess
import re
import pandas as pd

if(steps[0]):
    steps[0] = False
    instructor = input("Enter the username for instructor: ")
    instructor_lower = instructor.lower()
    password = input("Enter the password: ")
    course = input("Enter course name: ")
    course_lower = course.lower()

    data = pd.DataFrame([instructor_lower], [course_lower])
    data.to_csv("data.csv")

    # Add user
    add_user_cmd = ["sudo", "-S", "useradd", "-m", instructor_lower]
    passwd_cmd = ["sudo", "-S", "passwd", instructor_lower]

    sudo_password = "12345"  # Replace with your actual sudo password

    add_user_proc = subprocess.Popen(add_user_cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    add_user_proc.stdin.write(f"{sudo_password}\n".encode())
    add_user_proc.stdin.flush()



    # Wait for the command to complete and capture the output
    add_user_output, add_user_error = add_user_proc.communicate()
    # print(add_user_output.decode())
    # print(add_user_error.decode())

    passwd_proc = subprocess.Popen(passwd_cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    passwd_proc.stdin.write(f"{sudo_password}\n{password}\n{password}\n".encode())
    passwd_proc.stdin.flush()

    # Wait for the command to complete and capture the output
    passwd_output, passwd_error = passwd_proc.communicate()
    # print(passwd_output.decode())
    # print(passwd_error.decode())
    subprocess.run(["sudo", "usermod", "--shell", "/bin/bash", instructor_lower])
    chown_cmd = ["sudo", "chown", f"{instructor_lower}:{instructor_lower}", f"/home/{instructor_lower}"]
    chmod_cmd = ["sudo", "chmod", "777", f"/home/{instructor_lower}/"]
    subprocess.run(chown_cmd)
    subprocess.run(chmod_cmd)



    command = ['sudo', 'passwd', instructor_lower]
    # Open a subprocess to execute the command
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    # process.stdin.write(f"{sudo_password}\n".encode())
    # Send the new password to the passwd command's standard input
    process.stdin.write(password + '\n')
    process.stdin.write(password + '\n')
    process.stdin.flush()



    source = 'data.csv'
    source1 = 'instructor-script.ipynb'
    source3 = "Assignment3.ipynb"
    source4 = "solutions.zip"
    destination = f'/home/{instructor_lower}/'

    subprocess.call(['cp', source, destination])
    subprocess.call(['cp', source1, destination])
    subprocess.call(['cp', source3, destination])
    subprocess.call(['cp', source4, destination])





    # Open the config file for reading and writing
    with open("/opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py", "r+") as f:
        # Read the entire file content
        content = f.read()
        # Find the line that sets the admin_users property
        for line in content.split("\n"):
            if line.startswith("c.Authenticator.admin_users = {"):
                # Get the current set of admin users
                admin_users = line.split("{")[1].split("}")[0].strip()
                # Add the new user to the set of admin users
                admin_users += f", '{instructor_lower}'"
                # Replace the line with the updated admin_users set
                content = content.replace(line, f"c.Authenticator.admin_users = {{{admin_users}}}")
                # Write the updated content back to the file
                f.seek(0)
                f.write(content)
                print(f"Instructor {instructor} Added for course {course}")
                f.truncate()
                break
else:
    print("\nYou already executed this step!\n")


# Script to add students in JupyterHub
# To create their separate accounts

In [ ]:
# Run this cell to add students
import subprocess
import re

if(steps[1]):
    
    steps[1] = False
    
    #Array of Users which will be taken from LMS
    students = ['bc180402110','bc180402112','bc190202025','bc190202026','bc190204547','bc190204548','bc190305523','bc190305599']
    password = "12345"

    # Add students
    for user in students:

        # Add user
        add_user_cmd = ["sudo", "-S", "useradd", "-m", user]
        passwd_cmd = ["sudo", "-S", "passwd", user]

        sudo_password = "12345"  # Replace with your actual sudo password

        add_user_proc = subprocess.Popen(add_user_cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        add_user_proc.stdin.write(f"{sudo_password}\n".encode())
        add_user_proc.stdin.flush()

        # Wait for the command to complete and capture the output
        add_user_output, add_user_error = add_user_proc.communicate()
        # print(add_user_output.decode())
        # print(add_user_error.decode())

        passwd_proc = subprocess.Popen(passwd_cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        passwd_proc.stdin.write(f"{sudo_password}\n{password}\n{password}\n".encode())
        passwd_proc.stdin.flush()

        # Wait for the command to complete and capture the output
        passwd_output, passwd_error = passwd_proc.communicate()
        # print(passwd_output.decode())
        # print(passwd_error.decode())
        subprocess.run(["sudo", "usermod", "--shell", "/bin/bash", user])



        # subprocess.run(["sudo", "useradd", "-m", "-p", password, user])
        # Open the config file for reading and writing
        with open("/opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py", "r+") as f:
            # Read the entire file content
            content = f.read()
            # Find the line that sets the admin_users property
            for line in content.split("\n"):
                if line.startswith("c.Authenticator.allowed_users = {"):
                    # Get the current set of admin users
                    allowed_users = line.split("{")[1].split("}")[0].strip()
                    # Add the new user to the set of admin users
                    allowed_users += f", '{user}'"
                    # Replace the line with the updated admin_users set
                    content = content.replace(line, f"c.Authenticator.allowed_users = {{{allowed_users}}}")
                    # Write the updated content back to the file
                    f.seek(0)
                    f.write(content)
                    f.truncate()
                    break

    print("The following students are added")

    for student in students:
        print(student)
        
else:
    print("\nYou already executed this step!\n")


# Adding students to JupyterHUb Config file
# Assigning their respective Course

In [ ]:
if(steps[2]):
    steps[2] = False
    # Open the configuration file for reading
    with open('/opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py', 'r') as config_file:
        config_lines = config_file.readlines()

    # Find the line that defines the load_groups dictionary
    for i, line in enumerate(config_lines):
        if line.startswith('c.JupyterHub.load_groups = {'):
            start_line = i
            end_line = start_line + 1
            while end_line < len(config_lines) and '}' not in config_lines[end_line]:
                end_line += 1
            break

    newList = students.copy()
    newList.append(instructor_lower)



    # Add the new key-value pair to the dictionary
    config_lines.insert(end_line - 1, f"\t\t,'formgrade-{course_lower}': {newList},\n\t\t'nbgrader-{course_lower}':[]\n\n")

    # Write the updated configuration file
    with open('/opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py', 'w') as config_file:
        config_file.writelines(config_lines)



    # Open the configuration file for reading
    with open('/opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py', 'r') as config_file:
        config_lines = config_file.readlines()

    # Find the line that defines the load_groups dictionary
    for i, line in enumerate(config_lines):
        if line.startswith('c.JupyterHub.load_roles = ['):
            start_line = i
            end_line = start_line + 1
            while end_line < len(config_lines) and '{' not in config_lines[end_line]:
                end_line += 1
            break


    # Add the new key-value pair to the dictionary
    config_lines.insert(end_line - 1, f"\n\n\t{{\n\t\t'name': 'formgrader-{course_lower}-role',\n\t\t'groups': ['formgrade-{course_lower}'],\n\t\t'scopes': ['access:services!service={course_lower}']\n\t}},\n")

    # Write the updated configuration file
    with open('/opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py', 'w') as config_file:
        config_file.writelines(config_lines)

    print(f"{course} course assigned to students.")
    
else:
    print("\nYou already executed this step!\n")

# To automate JupyterHub Services

In [ ]:
if(steps[3]):
    steps[3] = False
    # Open the configuration file for reading
    with open('/opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py', 'r') as config_file:
        config_lines = config_file.readlines()

    # Find the line that defines the load_groups dictionary
    for i, line in enumerate(config_lines):
        if line.startswith('c.JupyterHub.services = ['):
            start_line = i
            end_line = start_line + 1
            while end_line < len(config_lines) and '{' not in config_lines[end_line]:
                end_line += 1
            break


    # Add the new key-value pair to the dictionary
    config_lines.insert(end_line - 1, f"\n\n\t{{\n\t\t'name': '{course_lower}',\n\t\t'url': 'http://127.0.0.1:8000',\n\t\t'command': ['jupyterhub-singleuser',\n\t\t'--group=formgrade-{course_lower}',\n\t\t'--debug'],\n\t\t'user': '{instructor_lower}',\n\t\t'cwd': '/home/{instructor_lower}'\n\t}},\n")

    # Write the updated configuration file
    with open('/opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py', 'w') as config_file:
        config_file.writelines(config_lines)

        print("Updated the services of JupyterHub to support newly created Users/Courses")
        
else:
    print("\nYou already executed this step!\n")

# Restart JupyterHub Server


In [ ]:
# To restart the server 
import pexpect

command = 'sudo systemctl restart jupyterhub.service'

child = pexpect.spawn(command, encoding='utf-8')
child.expect('password')
child.sendline('12345')
output = child.read().strip()
print(output)
